In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
# load submit data csv file
df_user_submit = pd.read_csv('./df_user_submit.csv')
df_user_submit.head()

,user_id,item_id,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_18
0,10001082,285259775,4.0,0.0,1.0,0.0,0.0,0.0
1,10001082,4368907,4.0,0.0,2.0,0.0,0.0,0.0
2,10001082,53616768,4.0,2.0,5.0,0.0,0.0,0.0
3,10001082,151466952,4.0,0.0,3.0,0.0,0.0,0.0
4,10001082,290088061,4.0,0.0,2.0,0.0,0.0,0.0


In [3]:
df_user_submit_data = df_user_submit[['count_user_buy','count_item_bought','count_look','count_favor','count_addcart','count_addcart_18']]

# extract only features and transform to ndarray 
df_user_submit_data = np.array(df_user_submit_data)

In [4]:
# load the scaler
scaler = pickle.load(open("./scaler", "rb"))
df_user_submit_data = scaler.transform(df_user_submit_data)
df_user_submit_data[0]

array([-0.68483934, -0.21403758, -0.60331779, -0.2220388 , -0.23288688,
       -0.01667692])

In [5]:
# load the svm
svm_classifier = pickle.load(open("./svm_classifier", "rb"))
# Get the prediction and transform to Dataframe
df_user_submit_data_prediction = svm_classifier.predict(df_user_submit_data)
df_prediction = pd.Series(df_user_submit_data_prediction)
# combine the prediction and the original csv file
df_user_submit['prediction'] = df_prediction
df_user_submit.head()

,user_id,item_id,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_18,prediction
0,10001082,285259775,4.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10001082,4368907,4.0,0.0,2.0,0.0,0.0,0.0,0.0
2,10001082,53616768,4.0,2.0,5.0,0.0,0.0,0.0,0.0
3,10001082,151466952,4.0,0.0,3.0,0.0,0.0,0.0,0.0
4,10001082,290088061,4.0,0.0,2.0,0.0,0.0,0.0,0.0


In [6]:
# select those user-item pair only if prediction =1
df_user_submit = df_user_submit.loc[df_user_submit['prediction']==1.0]
df_user_submit = df_user_submit[['user_id','item_id']]
df_user_submit.head()

,user_id,item_id
21,10001082,266188611
107,100029775,214252945
113,100029775,238681741
238,100068031,254127882
260,100068031,97781635


In [8]:
df_user_submit.shape

(81104, 2)

In [9]:
# 最后取一个交集，去掉不在商品自己里的预测数据
df_item_subset = pd.read_csv('./tianchi_fresh_comp_train_item.csv')
df_item_subset = df_item_subset[['item_id']]

In [13]:
df_result = pd.merge(left=df_user_submit, right=df_item_subset, how='inner', on=['item_id'])
df_result.head()

,user_id,item_id
0,100068031,254127882
1,100068031,97781635
2,100068031,303580397
3,100068031,55633951
4,100076491,191963647


In [14]:
df_result.to_csv('./tianchi_mobile_recommendation_predict.csv', index=False, header=True, encoding='utf8')